In [1]:

import os
import sys
import uuid
import pyspark
import datetime
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from faker import Faker
import datetime
from datetime import datetime
import random
import pandas as pd  # Import Pandas library for pretty printing

print("Imports loaded ")

Imports loaded 


In [2]:
HUDI_VERSION = '0.14.0'
SPARK_VERSION = '3.4'

SUBMIT_ARGS = f"--packages org.apache.hudi:hudi-spark{SPARK_VERSION}-bundle_2.12:{HUDI_VERSION},org.apache.hadoop:hadoop-aws:3.3.2 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS
os.environ['PYSPARK_PYTHON'] = sys.executable

spark = SparkSession.builder \
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .config('spark.sql.extensions', 'org.apache.spark.sql.hudi.HoodieSparkSessionExtension') \
    .config('className', 'org.apache.hudi') \
    .config('spark.sql.hive.convertMetastoreParquet', 'false') \
    .getOrCreate()

spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000/")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "admin")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "password")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
spark._jsc.hadoopConfiguration().set(
    "fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider",
                                     "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")


In [5]:
db_name = "default"
InventoryPath = f"s3a://global-emart/hudi/database={db_name}/table_name=Inventory"
Order_ItemPath = f"s3a://global-emart/hudi/database={db_name}/table_name=Order_Item"
OrderPath = f"s3a://global-emart/hudi/database={db_name}/table_name=Order"
PaymentPath = f"s3a://global-emart/hudi/database={db_name}/table_name=Payment"
ProductPath = f"s3a://global-emart/hudi/database={db_name}/table_name=Product"
ShipmentPath = f"s3a://global-emart/hudi/database={db_name}/table_name=Shipment"
UserPath = f"s3a://global-emart/hudi/database={db_name}/table_name=User"

# UPSERT
Scenario: Updating the price of Wireless Headphones.

Description: Sarah Johnson updates the price of her listed product, Wireless Headphones, from $150.00 to $140.00. The existing record in the product table is updated with the new price.

Operation: Upsert

In [4]:
existing_schema_df = spark.read.format("hudi").load(ProductPath)
existing_schema_df.printSchema()

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- listing_date: string (nullable = true)



In [19]:
# Data for the updated product
data = [
    ('1', 'Wireless Headphones', 'Noise-cancelling wireless headphones', 'Electronics', '140.00', '1003', '2024-05-28')
]

columns = ["product_id", "name", "description", "category", "price", "seller_id", "listing_date"]
product_df = spark.createDataFrame(data, columns)
product_df.printSchema()
def upsert_to_hudi(spark_df,
                  table_name,
                  db_name,
                  method='upsert',
                  table_type='COPY_ON_WRITE'
                  ):
    path = f"s3a://huditest/hudi/database={db_name}/table_name={table_name}"

    hudi_options = {
        'hoodie.table.name': table_name,
        'hoodie.datasource.write.table.type': table_type,
        'hoodie.datasource.write.table.name': table_name,
        'hoodie.datasource.write.operation': method,
        'hoodie.datasource.write.precombine.field': 'product_id',
        'hoodie.datasource.write.recordkey.field': 'product_id',

        "hoodie.datasource.hive_sync.database": db_name,
        "hoodie.datasource.hive_sync.table": table_name,
        "hoodie.datasource.hive_sync.metastore.uris": "thrift://localhost:9083",
        "hoodie.datasource.hive_sync.mode": "hms",
        "hoodie.datasource.hive_sync.enable": "true",
    }

    print("\n")
    print(path)
    print("\n")

    spark_df.write.format("hudi"). \
        options(**hudi_options). \
        mode("append"). \
        save(path)

try:
    upsert_to_hudi(
        spark_df=product_df,
        db_name="default",
        table_name="Product"
    )
    print("Upserted")

except Exception as e:
    print("error", e)

root
 |-- product_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- listing_date: string (nullable = true)



s3a://huditest/hudi/database=default/table_name=Product


error An error occurred while calling o307.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 105.0 failed 1 times, most recent failure: Lost task 0.0 in stage 105.0 (TID 442) (host.docker.internal executor driver): org.apache.hudi.exception.HoodieUpsertException: Error upserting bucketType UPDATE for partition :0
	at org.apache.hudi.table.action.commit.BaseSparkCommitActionExecutor.handleUpsertPartition(BaseSparkCommitActionExecutor.java:342)
	at org.apache.hudi.table.action.commit.BaseSparkCommitActionExecutor.lambda$mapPartitionsAsRDD$a3ab3c4$1(BaseSparkCommitActionExecutor.java:257)
	at org.apache.sp

In [9]:
ProductsDF = spark.read.format("hudi").load(ProductPath)
# tripsDF.createOrReplaceTempView("trips_table")
ProductsDF.show()

+-------------------+--------------------+--------------------+----------------------+--------------------+----------+-------------+--------------------+---------+-----+---------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|product_id|         name|         description| category|price|seller_id|       listing_date|
+-------------------+--------------------+--------------------+----------------------+--------------------+----------+-------------+--------------------+---------+-----+---------+-------------------+
|  20240531111427447|20240531111427447...|20240531111427447...|                      |0ec1b41f-7437-4c3...|         1|        Child|Turn last into re...|    sport|  638|     2536|2023-08-09 18:59:42|
|  20240531111427447|20240531111427447...|20240531111427447...|                      |0ec1b41f-7437-4c3...|         2|        Cover|As identify per d...|      job|  865|     2792|2023-03-07 20:04:31|


# Insert

Scenario: Adding a new user, Sarah Johnson.

Description: A new user, Sarah Johnson, registers on Global EMart. A new record is added to the user table with her details.

Operation: Insert

In [9]:
existing_schema_df = spark.read.format("hudi").load(UserPath)
existing_schema_df.printSchema()
existing_schema_df.show(5)
existing_schema_df.tail(5)

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: string (nullable = true)
 |-- last_login_date: string (nullable = true)

+-------------------+--------------------+--------------------+----------------------+--------------------+-------+---------------+--------------------+--------------------+-----------------+-------------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|user_id|           name|               email|             country|registration_date|    last_login_date|
+-------------------+--------------------+--------------------+----

[Row(_hoodie_commit_time='20240603164649566', _hoodie_commit_seqno='20240603164649566_0_4996', _hoodie_record_key='20240603164649566_0_4996', _hoodie_partition_path='', _hoodie_file_name='9d79cafa-f350-4cc4-830c-9413fbc3b1f7-0_0-4-49_20240603171423045.parquet', user_id='4997', name='Steven Young', email='scott07@example.net', country='Guam', registration_date='1966-04-03', last_login_date='2023-02-24 12:48:16'),
 Row(_hoodie_commit_time='20240603164649566', _hoodie_commit_seqno='20240603164649566_0_4997', _hoodie_record_key='20240603164649566_0_4997', _hoodie_partition_path='', _hoodie_file_name='9d79cafa-f350-4cc4-830c-9413fbc3b1f7-0_0-4-49_20240603171423045.parquet', user_id='4998', name='Isaiah Cook', email='jennifercastro@example.net', country='Sierra Leone', registration_date='1944-12-20', last_login_date='2022-11-13 06:04:05'),
 Row(_hoodie_commit_time='20240603164649566', _hoodie_commit_seqno='20240603164649566_0_4998', _hoodie_record_key='20240603164649566_0_4998', _hoodie_part

In [27]:
# Data for the new user
data = [
    ('3', 'Sarah Johnson 2', 'sarah.johnson2@example.com', 'USA', '2024-05-28', '2024-05-28')
]

columns = ["user_id", "name", "email", "country", "registration_date", "last_login_date"]
user_df = spark.createDataFrame(data, columns)
user_df.printSchema()

def insert_to_hudi(spark_df,
                  table_name,
                  db_name,
                  method='insert',
                  table_type='COPY_ON_WRITE'
                  ):
    path = f"s3a://global-emart/hudi/database={db_name}/table_name={table_name}"

    hudi_options = {
        'hoodie.table.name': table_name,
        'hoodie.datasource.write.table.type': table_type,
        'hoodie.datasource.write.table.name': table_name,
        'hoodie.datasource.write.operation': method,

        "hoodie.datasource.hive_sync.database": db_name,
        "hoodie.datasource.hive_sync.table": table_name,
        "hoodie.datasource.hive_sync.metastore.uris": "thrift://localhost:9083",
        "hoodie.datasource.hive_sync.mode": "hms",
        "hoodie.datasource.hive_sync.enable": "true",
    }

    print("\n")
    print(path)
    print("\n")

    spark_df.write.format("hudi"). \
        options(**hudi_options). \
        mode("append"). \
        save(path)

insert_to_hudi(
    spark_df=user_df,
    db_name="default",
    table_name="User"
)

root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: string (nullable = true)
 |-- last_login_date: string (nullable = true)



s3a://global-emart/hudi/database=default/table_name=User




# BULK_INSERT

Scenario: Bulk insert of a large number of new orders during a sale event.

Description: During a major sales event, a large number of new orders are placed. These orders are inserted in bulk into the order table to efficiently handle the high volume of new data.

Operation: Bulk Insert

In [24]:
existing_schema_df = spark.read.format("hudi").load(OrderPath)
existing_schema_df.printSchema()

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_time: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [33]:
existing_df = spark.read.format("hudi").load("s3a://huditest/hudi/database=default/table_name=Order")
existing_df.printSchema()

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_time: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [10]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType

schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_date", StringType(), True),
    StructField("order_time", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("total_amount", StringType(), True)
])

# Create DataFrame with the explicit schema
order_data = [
    ('5001', '2024-05-29', '11:00:00', '1002', 200.00),
    ('5002', '2024-05-29', '11:05:00', '1003', 350.00),
    # Add more orders as needed
]
order_df = spark.createDataFrame(order_data, schema)
order_df.printSchema()


root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_time: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [13]:
def bulk_insert_to_hudi(spark_df, table_name, db_name, method='bulk_insert', table_type='COPY_ON_WRITE'):
    path = f"s3a://global-emart/hudi/database={db_name}/table_name={table_name}"

    hudi_options = {
        'hoodie.table.name': table_name,
        'hoodie.datasource.write.table.type': table_type,
        'hoodie.datasource.write.operation': method,
        'hoodie.datasource.write.recordkey.field': 'order_id',
        'hoodie.datasource.write.precombine.field': 'order_date',
        'hoodie.datasource.write.partitionpath.field': '',  # Add partition field if needed

        'hoodie.datasource.hive_sync.database': db_name,
        'hoodie.datasource.hive_sync.table': table_name,
        'hoodie.datasource.hive_sync.metastore.uris': "thrift://localhost:9083",
        'hoodie.datasource.hive_sync.mode': "hms",
        'hoodie.datasource.hive_sync.enable': "true",
        
        'hoodie.datasource.write.schema.evolution.enable': 'true',
        'hoodie.datasource.write.schema.evolution.validate': 'true'
    }

    print("\n")
    print(path)
    print("\n")

    spark_df.write.format("hudi"). \
        options(**hudi_options). \
        mode("append"). \
        save(path)

# Call the bulk_insert_to_hudi function with the correct DataFrame
bulk_insert_to_hudi(
    spark_df=order_df,
    db_name="default",
    table_name="Order"
)




s3a://global-emart/hudi/database=default/table_name=Order




Py4JJavaError: An error occurred while calling o132.save.
: org.apache.hudi.exception.HoodieException: Config conflict(key	current value	existing value):
RecordKey:	order_id	null
	at org.apache.hudi.HoodieWriterUtils$.validateTableConfig(HoodieWriterUtils.scala:211)
	at org.apache.hudi.HoodieSparkSqlWriter$.writeInternal(HoodieSparkSqlWriter.scala:177)
	at org.apache.hudi.HoodieSparkSqlWriter$.write(HoodieSparkSqlWriter.scala:132)
	at org.apache.hudi.DefaultSource.createRelation(DefaultSource.scala:150)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:47)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:84)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eage

# DELETE

Scenario: Bob cancels his order for the Wireless Headphones.

Description: Bob decides to cancel his order for the Wireless Headphones. The corresponding record is deleted from the order table.

Operation: Delete

In [15]:
# Data for the canceled order (only record keys needed)
data = [
    ('1','2023-08-19','20:35:04','1210','3565.53')
]

columns = ["order_id", "order_date", "order_time", "customer_id", "total_amount"]
order_df = spark.createDataFrame(data, columns)

# Hudi configuration
hudi_options = {
    'hoodie.table.name': 'Order',
    'hoodie.datasource.write.recordkey.field': 'order_id',
    'hoodie.datasource.write.precombine.field': 'order_date',
    'hoodie.datasource.write.operation': 'delete',
}

# Write to Hudi table
order_df.write.format("hudi").options(**hudi_options).mode("append").save(OrderPath)


Py4JJavaError: An error occurred while calling o162.save.
: org.apache.hudi.exception.HoodieException: Config conflict(key	current value	existing value):
RecordKey:	order_id	null
	at org.apache.hudi.HoodieWriterUtils$.validateTableConfig(HoodieWriterUtils.scala:211)
	at org.apache.hudi.HoodieSparkSqlWriter$.writeInternal(HoodieSparkSqlWriter.scala:177)
	at org.apache.hudi.HoodieSparkSqlWriter$.write(HoodieSparkSqlWriter.scala:132)
	at org.apache.hudi.DefaultSource.createRelation(DefaultSource.scala:150)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:47)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:73)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:84)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eage

In [14]:
existing_df = spark.read.format("hudi").load(OrderPath)
existing_df.printSchema()
existing_df.show()

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_time: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- total_amount: string (nullable = true)

+-------------------+--------------------+--------------------+----------------------+--------------------+--------+----------+----------+-----------+------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|order_id|order_date|order_time|customer_id|total_amount|
+-------------------+--------------------+--------------------+----------------------+--------------------+--------+----------+----------+-----------+------------+
|  20240603164629005|20240603164629

# INSERT_OVERWRITE

Scenario: Overwriting existing user data with new data for the same users.

Description: Existing user data for Alice Smith and Bob Brown needs to be updated with new details. The operation overwrites the existing records with the new data.

Operation: Insert Overwrite

In [31]:
# New data for existing users
data = [
    ('1', 'Emily Howard', 'jaredbooth@example.net', 'USA', '2023-01-10', '2024-05-28'),
    # ('2', 'James Conner', 'christopherburns@example.net', 'USA', '2023-02-15', '2024-05-28')
]

columns = ["user_id", "name", "email", "country", "registration_date", "last_login_date"]
user_df = spark.createDataFrame(data, columns)

# Hudi configuration
def insert_overwrite_to_hudi(spark_df,
                  table_name,
                  db_name,
                  method='insert_overwrite',
                  table_type='COPY_ON_WRITE'
                  ):
    path = f"s3a://global-emart/hudi/database={db_name}/table_name={table_name}"

    hudi_options = {
        'hoodie.table.name': table_name,
        'hoodie.datasource.write.table.type': table_type,
        'hoodie.datasource.write.table.name': table_name,
        'hoodie.datasource.write.operation': method,

        "hoodie.datasource.hive_sync.database": db_name,
        "hoodie.datasource.hive_sync.table": table_name,
        "hoodie.datasource.hive_sync.metastore.uris": "thrift://localhost:9083",
        "hoodie.datasource.hive_sync.mode": "hms",
        "hoodie.datasource.hive_sync.enable": "true",
    }

    print("\n")
    print(path)
    print("\n")

    spark_df.write.format("hudi"). \
        options(**hudi_options). \
        mode("overwrite"). \
        save(path)

insert_overwrite_to_hudi(
    spark_df=user_df,
    db_name="default",
    table_name="User"
)

# Write to Hudi table




s3a://global-emart/hudi/database=default/table_name=User




In [32]:
existing_df = spark.read.format("hudi").load(UserPath)
# existing_df.printSchema()
existing_df.show()

+-------------------+--------------------+--------------------+----------------------+--------------------+-------+------------+--------------------+-------+-----------------+---------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|user_id|        name|               email|country|registration_date|last_login_date|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------+------------+--------------------+-------+-----------------+---------------+
|  20240603173841641|20240603173841641...|20240603173841641...|                      |11093755-58ef-42a...|      1|Emily Howard|jaredbooth@exampl...|    USA|       2023-01-10|     2024-05-28|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------+------------+--------------------+-------+-----------------+---------------+



# INSERT_OVERWRITE_TABLE

Scenario: Overwriting the entire product table with new product listings.

Description: The entire product catalog is refreshed with new product listings. This operation overwrites all existing records in the product table with the new data.

Operation: Insert Overwrite Table

In [15]:
# New data for products
data = [
    ('1', 'Laptop', 'High-performance laptop', 'Electronics', '1200.00', '1001', '2024-05-28'),
    ('2', 'Smartphone', 'Latest model smartphone', 'Electronics', '800.00', '1002', '2024-05-28'),
    # Add more products as needed
]

columns = ["product_id", "name", "description", "category", "price", "seller_id", "listing_date"]
product_df = spark.createDataFrame(data, columns)

# Hudi configuration
hudi_options = {
    'hoodie.table.name': 'product_table',
    'hoodie.datasource.write.recordkey.field': 'product_id',
    'hoodie.datasource.write.precombine.field': 'listing_date',
    'hoodie.datasource.write.operation': 'insert_overwrite_table',
}

# Write to Hudi table
product_df.write.format("hudi").options(**hudi_options).mode("overwrite").save(ProductPath)


In [18]:
productDF = spark.read.format("hudi").load(ProductPath)
productDF.show()

+-------------------+--------------------+------------------+----------------------+--------------------+----------+----------+--------------------+-----------+-------+---------+------------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|product_id|      name|         description|   category|  price|seller_id|listing_date|
+-------------------+--------------------+------------------+----------------------+--------------------+----------+----------+--------------------+-----------+-------+---------+------------+
|  20240530203535673|20240530203535673...|                 1|                      |78c81db0-772c-42c...|         1|    Laptop|High-performance ...|Electronics|1200.00|     1001|  2024-05-28|
|  20240530203535673|20240530203535673...|                 2|                      |78c81db0-772c-42c...|         2|Smartphone|Latest model smar...|Electronics| 800.00|     1002|  2024-05-28|
+-------------------+-------------------

# DELETE_PARTITION

Scenario: Removing all records for users from the USA.

Description: All user records from the USA need to be removed from the user table. This operation deletes the entire partition corresponding to the country 'USA'.

Operation: Delete Partition

In [18]:
# Data for the partition to delete
data = [
    ('Ireland')
]

columns = ["country"]
partition_df = spark.createDataFrame(data, columns)

# Hudi configuration
hudi_options = {
    'hoodie.table.name': 'user_table',
    'hoodie.datasource.write.operation': 'delete_partition',
    'hoodie.datasource.write.partitionpath.field': 'country',
}

# Write to Hudi table
partition_df.write.format("hudi").options(**hudi_options).mode("append").save(UserPath)


PySparkTypeError: [CANNOT_INFER_SCHEMA_FOR_TYPE] Can not infer schema for type: `str`.